In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import time
import re

In [2]:
cosmetics = pd.read_csv(r'..\cosmetics.csv')
cosmetics.ChemicalName = cosmetics.ChemicalName.apply(lambda x: x.lower())
cosmetics.ProductName = cosmetics.ProductName.apply(lambda x: x.lower())
cosmetics.dropna(subset='CasNumber', inplace=True)
cosmetics.head()

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,...,CasNumber,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
0,2,ultra color rich extra plump lipstick-all shades,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),53,...,13463-67-7,6,titanium dioxide,06/17/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1
1,3,glover's medicated shampoo,NaN,NaN,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,...,65996-92-1,4,distillates (coal tar),07/01/2009,07/01/2009,NaN,07/01/2009,07/01/2009,NaN,2
2,3,glover's medicated shampoo,NaN,NaN,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,...,140-67-0,5,estragole,07/01/2009,07/01/2009,NaN,07/02/2009,07/02/2009,NaN,2
3,4,precision glimmer eye liner-all shades �,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),46,...,13463-67-7,7,titanium dioxide,07/09/2009,08/28/2013,NaN,07/09/2009,07/09/2009,NaN,1
4,5,avon brilliant shine lip gloss-all shades �,NaN,NaN,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),52,...,13463-67-7,8,titanium dioxide,07/09/2009,08/28/2013,02/01/2011,07/09/2009,07/09/2009,NaN,1


In [3]:
print(cosmetics.shape)

print(cosmetics.dropna(subset=['CasNumber', 'CSF']).shape)

(108159, 22)
(76759, 22)


# Transformar a diccionario para hacer las correcciones

In [4]:
# Modificacion de algunas entradas para que sea más facil extraer la info de EWG
cosmetics.loc[cosmetics['ChemicalName']=='o-phenylenediamine and its salts', 'ChemicalName'] = '4-Nitro-O-Phenylenediamine'
cosmetics.loc[cosmetics['ChemicalName']=='distillates (coal tar)', 'ChemicalName'] = 'coal tar'
cosmetics.loc[cosmetics['ChemicalName']=='retinol/retinyl esters, when in daily dosages in excess of 10,000 iu, or 3,000 retinol equivalents.', 'ChemicalName'] = 'retinyl palmitate'
cosmetics.loc[cosmetics['ChemicalName']=='vitamin a palmitate', 'ChemicalName'] = 'retinyl palmitate'
cosmetics.loc[cosmetics['ChemicalName'] == 'carbon black (airborne, unbound particles of respirable size)', 'ChemicalName'] = 'carbon black'
cosmetics.loc[cosmetics['ChemicalName'] == 'dichloromethane (methylene chloride)', 'ChemicalName'] = 'dichloromethane'

cosmetics.loc[cosmetics['ChemicalName'] == 'cadmium and cadmium compounds', 'ChemicalName'] ='candium'
cosmetics.loc[cosmetics['ChemicalName'] == 'cadmium', 'CasNumber'] = '7440-43-9'
cosmetics.loc[cosmetics['ChemicalName'] == 'cocamide diethanolamine', 'CasNumber'] = '68603-42-9'
cosmetics.loc[cosmetics['ChemicalName'] == 'carbon black', 'CasNumber'] = '1333-86-4'
cosmetics.loc[cosmetics['ChemicalName'] == 'methylene glycol', 'CasNumber'] = '463-57-0'

cosmetics.loc[cosmetics['ChemicalName'] == 'selenium sulfide', 'CasNumber'] = '7488-56-4'

cosmetics.loc[cosmetics['ChemicalName'] == 'quinoline and its strong acid salts', 'ChemicalName'] ='quinoline'
cosmetics.loc[cosmetics['ChemicalName']=='mercury and mercury compounds', 'ChemicalName'] = 'mercury'
cosmetics.loc[cosmetics['ChemicalName']=='coal tar extract', 'ChemicalName'] = 'coal tar'

cosmetics.loc[cosmetics['ChemicalName']=='retinyl palmitate', 'ChemicalName'] = 'retinyl palmitate (vitamin a)'
cosmetics.loc[cosmetics['ChemicalName']=='vitamin a', 'ChemicalName'] = 'retinyl palmitate (vitamin a)'

cosmetics.loc[cosmetics['ChemicalName']=='acetic acid, retinyl ester', 'ChemicalName'] = 'retinyl acetate'
cosmetics.loc[cosmetics['ChemicalName']=='lauramide diethanolamine' , 'ChemicalName'] = 'lauramide dea'
cosmetics.loc[cosmetics['ChemicalName']=='cocamide diethanolamine' , 'ChemicalName'] = 'cocamide dea'
cosmetics.loc[cosmetics['ChemicalName']=='mineral oils, untreated and mildly treated' , 'ChemicalName'] = 'mineral oil'

cosmetics.loc[cosmetics['ChemicalName']=='bisphenol a (bpa)' , 'ChemicalName'] = 'bisphenol a'
cosmetics.loc[cosmetics['ChemicalName']=='ethanol in alcoholic beverages', 'ChemicalName'] = 'ethanol'
cosmetics.loc[cosmetics['ChemicalName']=='benzophenone‑4', 'ChemicalName'] = 'sulisobenzone'
cosmetics.loc[cosmetics['ChemicalName']=='benzophenone‑3', 'ChemicalName'] = 'oxybenzone'
cosmetics.loc[cosmetics['ChemicalName']=='all-trans retinoic acid', 'ChemicalName'] = 'retinoic acid'

cosmetics.loc[cosmetics['ChemicalName']=='formaldehyde (gas)', 'ChemicalName'] = 'formaldehyde'
cosmetics.loc[cosmetics['ChemicalName']=='genistein (purified)', 'ChemicalName'] ='genistein'
cosmetics.loc[cosmetics['ChemicalName']=='n-nitrosodimethylamine', 'ChemicalName'] ='dimethylnitrosamine'

cosmetics.loc[cosmetics['ChemicalName']=='di-n-butyl phthalate (dbp)', 'ChemicalName'] ='dibutyl phthalate'
cosmetics.loc[cosmetics['ChemicalName']=='quartz', 'ChemicalName'] ='crystalline silica'
cosmetics.loc[cosmetics['ChemicalName']=='propylene glycol mono-t-butyl ether', 'ChemicalName'] ='octinoxate'
cosmetics.loc[cosmetics['ChemicalName']=='2,2-bis(bromomethyl)-1,3-propanediol', 'ChemicalName'] ='tetramethyl decynediol'









In [5]:
chemicals = pd.read_csv(r'..\CTD_chemicals_diseases.csv')
chemicals.ChemicalName = chemicals.ChemicalName.apply(lambda x: x.lower())
print(chemicals.shape)
chemicals.head()

(9468010, 10)


,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedID
0,06-paris-la-66 protocol,C046983,NaN,Precursor Cell Lymphoblastic Leukemia-Lymphoma,MESH:D054198,therapeutic,NaN,NaN,NaN,4519131
1,10074-g5,C534883,NaN,Adenocarcinoma,MESH:D000230,NaN,MYC,4.08,NaN,26432044
2,10074-g5,C534883,NaN,Adenocarcinoma of Lung,MESH:D000077192,NaN,MYC,4.31,NaN,26656844|27602772
3,10074-g5,C534883,NaN,Alopecia,MESH:D000505,NaN,AR,4.52,NaN,15902657
4,10074-g5,C534883,NaN,Androgen-Insensitivity Syndrome,MESH:D013734,NaN,AR,6.89,300068|312300,1303262|8281139


In [6]:
# primero reviso en cosmetics los quimicos que tengan mas de 1 CasNumber
# Uso chemicals.CasRN para determinar cual es el CasNumber correcto
# reviso si hay productos que tengan igual CasNumber -> Si es así escojo un nombre para el químico que sea fácil de buscar en EWG

chem_cos = cosmetics[['CDPHId', 'ProductName', 'ChemicalName', 'CasNumber']].drop_duplicates()

for chemical in chem_cos.ChemicalName.unique():
    chemical_casRN = chem_cos.loc[chem_cos['ChemicalName']==chemical, 'CasNumber'].unique()
    cas_num = chem_cos.loc[chem_cos['ChemicalName']==chemical, 'CasNumber'].nunique()

    if cas_num > 1:
        print(chemical, ': ', cas_num)

        for CasN in chemical_casRN:
            if CasN in chemicals.CasRN.unique():
                chem_cos.loc[chem_cos['ChemicalName']==chemical, 'CasNumber'] = CasN
                # print(chemicals.loc[chemicals['CasRN']== CasN, 'DiseaseName'].drop_duplicates())
                # print('nuevo CasRN ', CasN)
    # print('----------------')


titanium dioxide :  10
coal tar :  3
retinyl palmitate (vitamin a) :  16
methyleugenol :  3
formaldehyde :  2
talc :  2
4-Nitro-O-Phenylenediamine :  2
p-aminodiphenylamine :  2


In [7]:
# for chemical in chem_cos.CasNumber.unique():
#     chemical_casRN = chem_cos.loc[chem_cos['CasNumber']==chemical, 'ChemicalName'].unique()
#     cas_num = chem_cos.loc[chem_cos['CasNumber']==chemical, 'ChemicalName'].nunique()

#     if cas_num > 1:
#         print(chemical, ': ', cas_num)
#         print(chemical_casRN)
        

In [8]:
# for casN in chem_cos['CasNumber'].unique():
#     chem_name = chem_cos.loc[chem_cos['CasNumber']==casN, 'ChemicalName'].unique()
#     if len(chem_name) >1:
#         print(chem_name)

In [9]:
chem_cos[['CasNumber', 'ChemicalName']].nunique()

CasNumber       86
ChemicalName    86
dtype: int64

In [10]:
ingredients = chem_cos['ChemicalName'].unique()
ingredients

array(['titanium dioxide', 'coal tar', 'estragole', 'cocamide dea',
       'toluene', 'retinol', 'retinyl palmitate (vitamin a)',
       'butylated hydroxyanisole', 'coffea arabica extract',
       'lauramide dea', 'carbon black', 'progesterone', 'methyleugenol',
       'o-phenylphenol', 'acrylamide', 'formaldehyde',
       'ginkgo biloba extract', 'mica', 'ethylene glycol',
       'retinyl acetate', 'ethyl acrylate', 'trade secret', 'methanol',
       'genistein', 'diethanolamine', 'tea-lauryl sulfate',
       'lead acetate', 'talc', 'triethanolamine',
       '4-Nitro-O-Phenylenediamine', 'safrole', 'styrene', 'acetaldehyde',
       '1,4-dioxane', 'dichloroacetic acid', 'ethylene oxide', 'lead',
       'dichloromethane', 'benzene', 'benzyl chloride',
       'dimethylnitrosamine', 'propylene oxide', 'methyl chloride',
       'candium', 'n-methylpyrrolidone', 'dibutyl phthalate', 'quinoline',
       'methylene glycol', 'benzophenone', 'musk xylene', 'aspirin',
       'benzophenone-3', '

In [11]:
import difflib

# Configurar navegador
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

resultados = []
not_found = []

for ingr in ingredients:
    casN = cosmetics[cosmetics['ChemicalName']==ingr]
    try:

        search_term = urllib.parse.quote(ingr)
        search_url = f"https://www.ewg.org/skindeep/search/?search={search_term}&search_type=ingredients"
        print('Ingrediente ', ingr)
        print(search_url)
        driver.get(search_url)
        time.sleep(2)

        links = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/ingredients/']"))
            )
        if not links:
            raise Exception("No se encontraron enlaces de ingredientes.")

        if len(links) == 1:
            best_link = links[0]
            print("Solo un resultado encontrado. Usando directamente.")
        else:
            # Obtener texto de cada enlace
            link_texts = [link.text.strip() for link in links]
            match = difflib.get_close_matches(ingr.lower(), [t.lower() for t in link_texts], n=1, cutoff=0.4)
        
            if match:
                match_index = [t.lower() for t in link_texts].index(match[0])
                best_link = links[match_index]
                print(f"Coincidencia más cercana: {link_texts[match_index]}")
            else:
                raise Exception("No se encontró una coincidencia adecuada")

        # Hacer scroll y forzar el click con JS para evitar errores de intercepción
        driver.execute_script("arguments[0].scrollIntoView(true);", best_link)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", best_link)
        time.sleep(2)

        # Parsear página del ingrediente
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # with open(f"debug_page_{ingr}.html", "w", encoding="utf-8") as f:
        #     f.write(soup.prettify())

        # Obtener nombre
        name_tag = soup.find('h2', class_='product-name text-block')
        name = name_tag.text.strip() if name_tag else "N/A"
        print('Nombre encontrado:', name)

        # # Obtener hazard score de forma robusta
    # Buscar imagen del hazard score
        score_img = soup.find('img', class_='squircle')
        hazard_score = "N/A"

        if score_img and 'src' in score_img.attrs:
            match = re.search(r'score=(\d+)', score_img['src'])
            if match:
                hazard_score = match.group(1)

        print("Hazard score:", hazard_score)


        # Extraer preocupaciones desde la sección 'ingredient-concerns'
        concerns_section = soup.find('section', class_='ingredient-concerns')
        concerns = []

        if concerns_section:
            items = concerns_section.find_all('li', class_='concern')
            for item in items:
                level_div = item.find('div', class_='level')
                level_text = level_div.get_text(strip=True) if level_div else 'N/A'
                
                concern_text_div = item.find('div', class_='concern-text')
                concern_text = concern_text_div.get_text(strip=True) if concern_text_div else 'N/A'

                concerns.append(f"{concern_text}: {level_text}")
        else:
            concerns.append("No concerns found")

        print('concerns ', concerns)
                
        concern_dict = {}
        for c in concerns:
            if ":" in c:
                key, value = c.split(":", 1)
                concern_dict[key.strip()] = value.strip()

        # Construir diccionario final con nombre + concerns
        result_dict = {
            "ChemicalName": ingr,
            "name_EWG": name,
            'hazard_score': hazard_score
        }
        result_dict.update(concern_dict)  # Añadir preocupaciones como columnas

        # Agregar al resultado
        resultados.append(result_dict)

    except Exception as e:
        print(f"Error con {ingr}")
        not_found.append(ingr)
        resultados.append({
            "ChemicalName": ingr,
            "name_EWG": None,
            "hazard_score": None,
        })

    print('..............................')
driver.quit()

Ingrediente  titanium dioxide
https://www.ewg.org/skindeep/search/?search=titanium%20dioxide&search_type=ingredients
Coincidencia más cercana: TITANIUM DIOXIDE
Nombre encontrado: Titanium Dioxide
Hazard score: 1
concerns  ['Cancer: MODERATE', 'Allergies & Immunotoxicity: LOW', 'Developmental and Reproductive Toxicity: LOW', 'Use Restrictions: LOW']
..............................
Ingrediente  coal tar
https://www.ewg.org/skindeep/search/?search=coal%20tar&search_type=ingredients
Coincidencia más cercana: COAL TAR
Nombre encontrado: Coal Tar
Hazard score: 10
concerns  ['Cancer: HIGH', 'Allergies & Immunotoxicity: MODERATE', 'Developmental and Reproductive Toxicity: LOW_MODERATE', 'Use Restrictions: HIGH']
..............................
Ingrediente  estragole
https://www.ewg.org/skindeep/search/?search=estragole&search_type=ingredients
Coincidencia más cercana: ESTRAGOLE
Nombre encontrado: Estragole
Hazard score: 4
concerns  ['Cancer: MODERATE', 'Allergies & Immunotoxicity: MODERATE', 'De

In [12]:
df = pd.DataFrame(resultados)
df.to_csv('ewg_cosmetics_data.csv', index=False)

In [14]:
df = pd.read_csv('ewg_cosmetics_data.csv')
df

,ChemicalName,name_EWG,hazard_score,Cancer,Allergies & Immunotoxicity,Developmental and Reproductive Toxicity,Use Restrictions
0,titanium dioxide,Titanium Dioxide,1.0,MODERATE,LOW,LOW,LOW
1,coal tar,Coal Tar,10.0,HIGH,MODERATE,LOW_MODERATE,HIGH
2,estragole,Estragole,4.0,MODERATE,MODERATE,LOW,LOW
3,cocamide dea,Cocamide Dea,8.0,MODERATE,MODERATE,LOW,HIGH
4,toluene,Toluene,9.0,LOW,LOW,MODERATE,HIGH
...,...,...,...,...,...,...,...
81,trichloroacetic acid,Trichloroacetic Acid,6.0,LOW_MODERATE,LOW,LOW,HIGH
82,phenacemide,NaN,NaN,NaN,NaN,NaN,NaN
83,polygeenan,NaN,NaN,NaN,NaN,NaN,NaN
84,ethanol,Ethanol,1.0,LOW,LOW,LOW,LOW


In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['ChemicalName'])
chemicals = []
for i, l in enumerate(le.classes_):
    chemicals.append([i,l])

chem_id_df = pd.DataFrame(columns=['ChemID', 'ChemicalName'], data=chemicals)

In [16]:
# cosmetics_new = cosmetics[['CDPHId', 'ProductName', 'ChemicalName', 'CasNumber']]

cosmetics_new = chem_cos.merge(chem_id_df)
cosmetics_new = cosmetics_new.merge(df).drop_duplicates()
cosmetics_new.head()

,CDPHId,ProductName,ChemicalName,CasNumber,ChemID,name_EWG,hazard_score,Cancer,Allergies & Immunotoxicity,Developmental and Reproductive Toxicity,Use Restrictions
0,2,ultra color rich extra plump lipstick-all shades,titanium dioxide,13463-67-7,80,Titanium Dioxide,1.0,MODERATE,LOW,LOW,LOW
1,3,glover's medicated shampoo,coal tar,8007-45-2,23,Coal Tar,10.0,HIGH,MODERATE,LOW_MODERATE,HIGH
2,3,glover's medicated shampoo,estragole,140-67-0,33,Estragole,4.0,MODERATE,MODERATE,LOW,LOW
3,4,precision glimmer eye liner-all shades �,titanium dioxide,13463-67-7,80,Titanium Dioxide,1.0,MODERATE,LOW,LOW,LOW
4,5,avon brilliant shine lip gloss-all shades �,titanium dioxide,13463-67-7,80,Titanium Dioxide,1.0,MODERATE,LOW,LOW,LOW


In [17]:
cosmetics_new.to_csv('cosmetics_chem_concerns.csv', index=False)

In [18]:
cosmetics_new[cosmetics_new['ProductName']=='texture cream']



,CDPHId,ProductName,ChemicalName,CasNumber,ChemID,name_EWG,hazard_score,Cancer,Allergies & Immunotoxicity,Developmental and Reproductive Toxicity,Use Restrictions
3877,4004,texture cream,"1,4-dioxane",123-91-1,0,"5 Bromo 5 Nitro 1,3 Dioxane",6.0,LOW,HIGH,LOW,MODERATE
3878,4004,texture cream,acetaldehyde,75-07-0,2,Acetaldehyde,9.0,HIGH,LOW,LOW,HIGH
3879,4004,texture cream,benzene,71-43-2,8,Ethylbenzene,6.0,MODERATE,LOW,LOW_MODERATE,LOW
3880,4004,texture cream,benzyl chloride,100-44-7,14,Benzoyl Chloride,3.0,LOW,MODERATE,LOW,LOW
3881,4004,texture cream,ethylene oxide,75-21-8,37,Ethylene Oxide,10.0,HIGH,LOW,MODERATE,HIGH
3882,4004,texture cream,formaldehyde,30525-89-4,38,Formaldehyde,10.0,HIGH,MODERATE,LOW,HIGH
3883,4004,texture cream,dimethylnitrosamine,62-75-9,32,Dimethylnitrosoamine,8.0,MODERATE,LOW,LOW,HIGH
3884,4004,texture cream,propylene oxide,75-56-9,65,Propylene Oxide,8.0,MODERATE,LOW,LOW,HIGH


In [23]:
cosmetics[(cosmetics['ProductName']=='dry skin lotion')&(cosmetics['ChemicalName']=='titanium dioxide')]

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,...,CasNumber,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
37471,13074,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Monkey Butt,90,Skin Care Products,102,...,13463-67-7,20432,titanium dioxide,06/23/2010,02/11/2015,10/04/2012,06/23/2010,12/10/2014,NaN,1
37474,13076,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Shopko,90,Skin Care Products,102,...,13463-67-7,20439,titanium dioxide,06/23/2010,02/11/2015,10/04/2012,06/23/2010,12/10/2014,NaN,1
37478,13088,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",America's Choice,90,Skin Care Products,102,...,13463-67-7,20455,titanium dioxide,06/23/2010,02/11/2015,10/04/2012,06/23/2010,12/10/2014,NaN,1
37480,13090,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Meijer,90,Skin Care Products,102,...,13463-67-7,20461,titanium dioxide,06/23/2010,02/11/2015,10/04/2012,06/23/2010,12/10/2014,NaN,1
37482,13094,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Western Family,90,Skin Care Products,102,...,13463-67-7,20469,titanium dioxide,06/23/2010,02/11/2015,NaN,06/23/2010,12/10/2014,NaN,1
37486,13105,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Hill Country Essentials,90,Skin Care Products,102,...,13463-67-7,20489,titanium dioxide,06/23/2010,02/11/2015,NaN,06/23/2010,12/10/2014,NaN,1
58464,21092,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Equate,90,Skin Care Products,102,...,13463-67-7,32106,titanium dioxide,10/08/2013,02/11/2015,NaN,10/08/2013,12/10/2014,NaN,1
58594,21159,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Ahold,90,Skin Care Products,102,...,13463-67-7,32215,titanium dioxide,10/16/2013,02/11/2015,NaN,10/16/2013,12/10/2014,NaN,1
58595,21160,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Market Spa,90,Skin Care Products,102,...,13463-67-7,32216,titanium dioxide,10/16/2013,02/11/2015,NaN,10/16/2013,12/10/2014,NaN,1
58596,21161,dry skin lotion,NaN,NaN,615,"Vi-Jon, Inc.",Quality Choice,90,Skin Care Products,102,...,13463-67-7,32217,titanium dioxide,10/16/2013,02/11/2015,NaN,10/16/2013,12/10/2014,NaN,1


In [22]:
cosmetics_new[['ProductName', 'ChemicalName']].value_counts(ascending=False)

ProductName                          ChemicalName    
permanent cosmetics color            titanium dioxide    92
                                     carbon black        92
dry skin lotion                      titanium dioxide    55
deep moisturizing body wash          titanium dioxide    40
nutrisse color creme                 titanium dioxide    37
                                                         ..
faux glow lacquer                    titanium dioxide     1
faux fur                             titanium dioxide     1
faux cils mascara                    titanium dioxide     1
fault line                           titanium dioxide     1
z�ro d�faut correcteur (all shades)  titanium dioxide     1
Name: count, Length: 35072, dtype: int64